# 0.489 lgb

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [19]:
!mkdir -p GuodongDrive
!google-drive-ocamlfuse GuodongDrive
import os
os.chdir("GuodongDrive")
os.chdir("ccf/carsale")
!ls

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
0825ccf_carsale.csv	   ccf_car_sales_lgb.csv
0825ccf_car_sales_lgb.csv  evaluation_public.csv
baseline082601.ipynb	   evaluation_public_xishao.csv
baseline082701.ipynb	   GuodongDrive
baseline082702.ipynb	   ronghe_car_sales.csv
baseline082703.ipynb	   submit_example.csv
baseline1_copy.ipynb	   train_sales_data.csv
baseline1.ipynb		   train_search_data.csv
baseline2.ipynb		   train_user_reply_data.csv
ccf_car_sales082601.csv    Untitled0.ipynb
ccf_car_sales082602.csv    统计_car_sales.csv
ccf_car_sales.csv


In [0]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
import xgboost as xgb

In [0]:
train_sales_data = pd.read_csv('train_sales_data.csv')
train_search_data = pd.read_csv('train_search_data.csv')
train_user_reply_data = pd.read_csv('train_user_reply_data.csv')

test = pd.read_csv('evaluation_public.csv')

In [0]:
data = pd.concat([train_sales_data, test], ignore_index=True)
data = data.merge(train_search_data, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user_reply_data, 'left', on=['model', 'regYear', 'regMonth'])
data.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum
0,310000,SUV,NaN,NaN,3c974920a76ac9c1,上海,1,2016,292.0,1479.0,11.0,106.0
1,530000,SUV,NaN,NaN,3c974920a76ac9c1,云南,1,2016,466.0,1594.0,11.0,106.0
2,150000,SUV,NaN,NaN,3c974920a76ac9c1,内蒙古,1,2016,257.0,1479.0,11.0,106.0
3,110000,SUV,NaN,NaN,3c974920a76ac9c1,北京,1,2016,408.0,2370.0,11.0,106.0
4,510000,SUV,NaN,NaN,3c974920a76ac9c1,四川,1,2016,610.0,3562.0,11.0,106.0
5,340000,SUV,NaN,NaN,3c974920a76ac9c1,安徽,1,2016,206.0,1314.0,11.0,106.0
6,370000,SUV,NaN,NaN,3c974920a76ac9c1,山东,1,2016,503.0,3476.0,11.0,106.0
7,140000,SUV,NaN,NaN,3c974920a76ac9c1,山西,1,2016,236.0,1422.0,11.0,106.0
8,440000,SUV,NaN,NaN,3c974920a76ac9c1,广东,1,2016,3635.0,7182.0,11.0,106.0
9,450000,SUV,NaN,NaN,3c974920a76ac9c1,广西,1,2016,450.0,1163.0,11.0,106.0


In [0]:
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
del data['salesVolume'], data['forecastVolum']
data['bodyType'] = data['model'].map(train_sales_data.drop_duplicates('model').set_index('model')['bodyType'])
data['model'].head()

0    3c974920a76ac9c1
1    3c974920a76ac9c1
2    3c974920a76ac9c1
3    3c974920a76ac9c1
4    3c974920a76ac9c1
Name: model, dtype: object

In [0]:
data.head(10)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0,292.0
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0,466.0
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0,257.0
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0,408.0
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0,610.0
5,340000,SUV,0,3c974920a76ac9c1,安徽,1,2016,1314.0,11.0,106.0,206.0
6,370000,SUV,0,3c974920a76ac9c1,山东,1,2016,3476.0,11.0,106.0,503.0
7,140000,SUV,0,3c974920a76ac9c1,山西,1,2016,1422.0,11.0,106.0,236.0
8,440000,SUV,0,3c974920a76ac9c1,广东,1,2016,7182.0,11.0,106.0,3635.0
9,450000,SUV,0,3c974920a76ac9c1,广西,1,2016,1163.0,11.0,106.0,450.0


In [0]:
for i in ['bodyType', 'model']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))#用这个函数可以查看数据有多少个不同值。

data.head(20)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label
0,310000,0,0,0,上海,1,2016,1479.0,11.0,106.0,292.0
1,530000,0,0,0,云南,1,2016,1594.0,11.0,106.0,466.0
2,150000,0,0,0,内蒙古,1,2016,1479.0,11.0,106.0,257.0
3,110000,0,0,0,北京,1,2016,2370.0,11.0,106.0,408.0
4,510000,0,0,0,四川,1,2016,3562.0,11.0,106.0,610.0
5,340000,0,0,0,安徽,1,2016,1314.0,11.0,106.0,206.0
6,370000,0,0,0,山东,1,2016,3476.0,11.0,106.0,503.0
7,140000,0,0,0,山西,1,2016,1422.0,11.0,106.0,236.0
8,440000,0,0,0,广东,1,2016,7182.0,11.0,106.0,3635.0
9,450000,0,0,0,广西,1,2016,1163.0,11.0,106.0,450.0


zip() 函数用于将可迭代的对象作为参数，

将对象中对应的元素打包成一个个元组，

然后返回由这些元组组成的列表。

如果各个迭代器的元素个数不一致，则返回列表长度与最短的对象相同，

a = [1,2,3]

 b = [4,5,6]

 c = [4,5,6,7,8]

 zipped = zip(a,b)  # 打包为元组的列表

[(1, 4), (2, 5), (3, 6)]

zip(a,c)              # 元素个数与最短的列表一致

[(1, 4), (2, 5), (3, 6)]

In [0]:
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']

# 'popularity', 'carCommentVolum', 'newsReplyVolum','label'/

shift_feat = []

data['model_adcode'] = data['adcode'] + data['model']
data['model_adcode_mt'] = data['model_adcode'] * 100 + data['mt']
for i in [11]:
    i = i + 1
    shift_feat.append('shift_model_adcode_mt_label_{0}'.format(i))
    data['model_adcode_mt_{0}'.format(i)] = data['model_adcode_mt'] + i
    data_last = data[~data.label.isnull()].set_index('model_adcode_mt_{0}'.format(i))
    data['shift_model_adcode_mt_label_{0}'.format(i)] = data['model_adcode_mt'].map(data_last['label'])

shift_feat

['shift_model_adcode_mt_label_12']

In [0]:
data.head(10)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,mt,model_adcode,model_adcode_mt,model_adcode_mt_12,shift_model_adcode_mt_label_12
0,310000,0,0,0,上海,1,2016,1479.0,11.0,106.0,292.0,1,310000,31000001,31000013,NaN
1,530000,0,0,0,云南,1,2016,1594.0,11.0,106.0,466.0,1,530000,53000001,53000013,NaN
2,150000,0,0,0,内蒙古,1,2016,1479.0,11.0,106.0,257.0,1,150000,15000001,15000013,NaN
3,110000,0,0,0,北京,1,2016,2370.0,11.0,106.0,408.0,1,110000,11000001,11000013,NaN
4,510000,0,0,0,四川,1,2016,3562.0,11.0,106.0,610.0,1,510000,51000001,51000013,NaN
5,340000,0,0,0,安徽,1,2016,1314.0,11.0,106.0,206.0,1,340000,34000001,34000013,NaN
6,370000,0,0,0,山东,1,2016,3476.0,11.0,106.0,503.0,1,370000,37000001,37000013,NaN
7,140000,0,0,0,山西,1,2016,1422.0,11.0,106.0,236.0,1,140000,14000001,14000013,NaN
8,440000,0,0,0,广东,1,2016,7182.0,11.0,106.0,3635.0,1,440000,44000001,44000013,NaN
9,450000,0,0,0,广西,1,2016,1163.0,11.0,106.0,450.0,1,450000,45000001,45000013,NaN


In [0]:
num_feat = ['regYear'] + shift_feat
cate_feat = ['adcode', 'bodyType', 'model', 'regMonth']

features = num_feat + cate_feat
num_feat

['regYear', 'shift_model_adcode_mt_label_12']

In [0]:
# data['n_label'] = data['label'] / data.groupby('model')['label'].transform('mean')
train_idx = (data['mt'] <= 20)

valid_idx = (data['mt'].between(21, 24))

test_idx = (data['mt'] > 24)

In [0]:
data['model_weight'] = data.groupby('model')['label'].transform('mean')
data['n_label'] = data['label'] / data['model_weight']

In [0]:
data.head(10)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,mt,model_adcode,model_adcode_mt,model_adcode_mt_12,shift_model_adcode_mt_label_12,model_weight,n_label
0,310000,0,0,0,上海,1,2016,1479.0,11.0,106.0,292.0,1,310000,31000001,31000013,NaN,444.518939,0.656890
1,530000,0,0,0,云南,1,2016,1594.0,11.0,106.0,466.0,1,530000,53000001,53000013,NaN,444.518939,1.048324
2,150000,0,0,0,内蒙古,1,2016,1479.0,11.0,106.0,257.0,1,150000,15000001,15000013,NaN,444.518939,0.578153
3,110000,0,0,0,北京,1,2016,2370.0,11.0,106.0,408.0,1,110000,11000001,11000013,NaN,444.518939,0.917846
4,510000,0,0,0,四川,1,2016,3562.0,11.0,106.0,610.0,1,510000,51000001,51000013,NaN,444.518939,1.372270
5,340000,0,0,0,安徽,1,2016,1314.0,11.0,106.0,206.0,1,340000,34000001,34000013,NaN,444.518939,0.463422
6,370000,0,0,0,山东,1,2016,3476.0,11.0,106.0,503.0,1,370000,37000001,37000013,NaN,444.518939,1.131560
7,140000,0,0,0,山西,1,2016,1422.0,11.0,106.0,236.0,1,140000,14000001,14000013,NaN,444.518939,0.530911
8,440000,0,0,0,广东,1,2016,7182.0,11.0,106.0,3635.0,1,440000,44000001,44000013,NaN,444.518939,8.177379
9,450000,0,0,0,广西,1,2016,1163.0,11.0,106.0,450.0,1,450000,45000001,45000013,NaN,444.518939,1.012330


In [0]:
train_x = data[train_idx][features]
train_y = data[train_idx]['n_label']

valid_x = data[valid_idx][features]
valid_y = data[valid_idx]['n_label']

# test_x = data[test_idx][features]

lgb_model = lgb.LGBMRegressor(
    num_leaves=32, reg_alpha=1, reg_lambda=0.1, objective='mse',
    max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=np.random.randint(1000),
    n_estimators=5000, subsample=0.8, colsample_bytree=0.8,
)

lgb_model.fit(train_x, train_y, eval_set=[
    (valid_x, valid_y),
], categorical_feature=cate_feat, early_stopping_rounds=100, verbose=100)



data['pred_label'] = lgb_model.predict(data[features]) * data['model_weight']

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['adcode', 'bodyType', 'model', 'regMonth']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.297777
[200]	valid_0's l2: 0.254184
[300]	valid_0's l2: 0.243312
[400]	valid_0's l2: 0.242705
Early stopping, best iteration is:
[357]	valid_0's l2: 0.240136


In [0]:

def score(data, pred='pred_label', label='label', group='model'):
  
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred: list,
        label: [list, 'mean'],

    }).reset_index()

    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)



best_score = score(data[valid_idx])




0.5994896880072957


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [0]:
lgb_model.n_estimators = 1000

lgb_model.fit(data[~test_idx][features], data[~test_idx]['n_label'], categorical_feature=cate_feat)
data['forecastVolum'] = lgb_model.predict(data[features]) * data['model_weight']
sub = data[test_idx][['id']]
sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
sub.head(10)
#sub.to_csv('082703sub.csv', index=False)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['adcode', 'bodyType', 'model', 'regMonth']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


,id,forecastVolum
31680,1,288
31681,2,383
31682,3,207
31683,4,342
31684,5,434
31685,6,235
31686,7,445
31687,8,240
31688,9,2580
31689,10,384


In [0]:
evaluation_public_xishao = sub
evaluation_public_xishao.to_csv('evaluation_public_xishao.csv', index=False)



In [20]:
evaluation_public_1 = pd.read_csv("evaluation_public_xishao.csv")
evaluation_public_1.head(5)

,id,forecastVolum
0,1,288
1,2,383
2,3,207
3,4,342
4,5,434


In [21]:
evaluation_public_1.shape

(5280, 2)

In [22]:
evaluation_public_2 = pd.read_csv("统计_car_sales.csv")

evaluation_public_2.head(5)

,id,forecastVolum
0,1,289
1,2,373
2,3,153
3,4,288
4,5,398


In [23]:
evaluation_public_2.shape

(5280, 2)

In [24]:
evaluation_public_2['forecastVolum'] = evaluation_public_2['forecastVolum']*0.5 + evaluation_public_1['forecastVolum']*0.5
evaluation_public_2.head()

,id,forecastVolum
0,1,288.5
1,2,378.0
2,3,180.0
3,4,315.0
4,5,416.0


In [25]:
evaluation_public_2['forecastVolum'] = evaluation_public_2['forecastVolum'].round().astype(int)
evaluation_public_2.head()

,id,forecastVolum
0,1,288
1,2,378
2,3,180
3,4,315
4,5,416


In [0]:
evaluation_public_2.to_csv('xishao_car_sale.csv', index=False)

In [0]:

# def get_predict_w(model, data, label='label', feature=[], cate_feature=[], random_state=2018, n_splits=5,
#                   model_type='lgb'):
#     if 'sample_weight' not in data.keys():
#         data['sample_weight'] = 1
#     model.random_state = random_state
#     predict_label = 'predict_' + label
#     kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
#     data[predict_label] = 0
#     test_index = (data[label].isnull()) | (data[label] == -1)
#     train_data = data[~test_index].reset_index(drop=True)
#     test_data = data[test_index]
#
#     for train_idx, val_idx in kfold.split(train_data):
#         model.random_state = model.random_state + 1
#
#         train_x = train_data.loc[train_idx][feature]
#         train_y = train_data.loc[train_idx][label]
#
#         test_x = train_data.loc[val_idx][feature]
#         test_y = train_data.loc[val_idx][label]
#         if model_type == 'lgb':
#             try:
#                 model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
#                           # eval_metric='mae',
#                           categorical_feature=cate_feature,
#                           sample_weight=train_data.loc[train_idx]['sample_weight'],
#                           verbose=100)
#             except:
#                 model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
#                           eval_metric='mae',
#                           # categorical_feature=cate_feature,
#                           sample_weight=train_data.loc[train_idx]['sample_weight'],
#                           verbose=100)
#         elif model_type == 'ctb':
#             model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
#                       # eval_metric='mae',
#                       cat_features=cate_feature,
#                       sample_weight=train_data.loc[train_idx]['sample_weight'],
#                       verbose=100)
#         train_data.loc[val_idx, predict_label] = model.predict(test_x)
#         if len(test_data) != 0:
#             test_data[predict_label] = test_data[predict_label] + model.predict(test_data[feature])
#     test_data[predict_label] = test_data[predict_label] / n_splits
#     print(mse(train_data[label], train_data[predict_label]), train_data[predict_label].mean(),
#           test_data[predict_label].mean())
#     return pd.concat([train_data, test_data], sort=True, ignore_index=True), predict_label
#
#
# data, predict_label = get_predict_w(lgb_model, data, label='label',
#                                     feature=features, cate_feature=cate_feat,
#                                     random_state=2019, n_splits=5)
#
# # 自定义评价函数
# model = data[valid_idx]['model'].values
#
#
# def n_rmse(preds, labels):
#     valid_data = pd.DataFrame({
#         'label': labels,
#         'pred_label': preds,
#         'model': model
#     })
#     n_rmse = score(valid_data)
#     return 'n_rmse', n_rmse, True
#
# #
# # data['lgb'] = data[predict_label]
# #
# # data['forecastVolum'] = data['lgb'].apply(lambda x: 0 if x < 0 else x)
# # data[data.label.isnull()][['id', 'forecastVolum']].round().astype(int).to_csv(path + '/sub/sub.csv', index=False)
# #
# # ## 简单版本，预测下一个月。
# #
# #
# # #
# # # xgb_model = xgb.XGBRegressor(
# # #     max_depth=5,
# # #     learning_rate=0.01,
# # #     objective='reg:linear',
# # #     eval_metric='mae',
# # #     subsample=0.8, colsample_bytree=0.5,
# # #     n_estimators=3000,
# # #     reg_alpha=0.,
# # #     reg_lambda=0.001,
# # #     min_child_weight=50,
# # #     n_jobs=8,
# # #     seed=42,
# # # )
# # #
# # # data, predict_label = get_predict_w(xgb_model, data, label='temp_label',
# # #                                     feature=features, random_state=2018, n_splits=5)
# # #
# # # data['xgb'] = data[predict_label]
# # #
# # # ctb_params = {
# # #     'n_estimators': 10000,
# # #     'learning_rate': 0.02,
# # #     'random_seed': 4590,
# # #     'reg_lambda': 0.08,
# # #     'subsample': 0.7,
# # #     'bootstrap_type': 'Bernoulli',
# # #     'boosting_type': 'Plain',
# # #     'one_hot_max_size': 10,
# # #     'rsm': 0.5,
# # #     'leaf_estimation_iterations': 5,
# # #     'use_best_model': True,
# # #     'max_depth': 6,
# # #     'verbose': -1,
# # #     'thread_count': 4
# # # }
# # #
# # # ctb_model = ctb.CatBoostRegressor(**ctb_params)
# # #
# # # data, predict_label = get_predict_w(ctb_model, data, label='temp_label',
# # #                                     feature=features,
# # #                                     random_state=2019, n_splits=5, model_type='ctb')
# # #
# # # data['ctb'] = data[predict_label]
# # #
# # # data['t_label'] = data['lgb_mse'] * 0.4 + data['xgb_mse'] * 0.1 + data['ctb_mse'] * 0.4 + data['lgb_mae'] * 0.1